In [1]:
#http://www.ivl.disco.unimib.it/activities/food-recognition/

In [1]:
import json
import cv2
import numpy as np
from skimage.draw import polygon
import imageio
import os
from progressbar import ProgressBar
import shutil

In [2]:
json_path1 = ".../UNIMIB2016 Food Database/train.json"
json_path2 = ".../UNIMIB2016 Food Database/val.json"
json_path3 = ".../UNIMIB2016 Food Database/test.json"

json_path1 = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/UNIMIB2016 Food Database/train.json"
json_path2 = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/UNIMIB2016 Food Database/val.json"
json_path3 = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/UNIMIB2016 Food Database/test.json"

with open(json_path1) as f:
    data1 = json.load(f)
    
with open(json_path2) as f:
    data2 = json.load(f)
    
with open(json_path3) as f:
    data3 = json.load(f)

In [4]:
food = {}
food_file = {}

dat = [data1,data2,data3]

for dd in dat:
    data=dd
    for d in data:
        objects = data[d]
        for ob in objects:
            for food_name in ob:
                if food_name not in food:
                    food[food_name] = 1
                    food_file[food_name] = [d]
                else:
                    food[food_name] = food[food_name] + 1
                    food_file[food_name].append(d)


In [1]:
#food_file['mandarini']

In [2]:
#{'bread':479,'pasta':566,'chicken':93,'carrots':161,'yogurt':130,'pizza':89,'cutlet':148,'tangerines':198,'mashed potato':151,'spinach':110,'pudding':112}

In [12]:
seg_coord = {}
classes = []

for d in data:
    objects = data[d]
    
    food = {}
    food_list = []
    for ob in objects:
        
        coor = {}
        for food_name in ob:

            if food_name not in classes:
                classes.append(food_name)
            
            BR = ob[food_name]['BR']
            BB = ob[food_name]['BB']
            
            w = []
            h = []

            for i in range(len(BR )):
                if i%2 != 0:
                    w.append(BR[i])
                else:
                    h.append(BR[i])   
            
            coor['w'] = w
            coor['h'] = h
            
            food[food_name] = coor
            
    food_list.append(food)
     
    seg_coord[d+'.jpg'] = food_list


In [5]:
classes_en = ['bread','pasta','scallops','carrots','yogurt','pizza','cutlet','tangerines','potatoes/mashed potato','green beans', 'spinach','pudding']
#we can use bread, pasta, carrots, pizza, tangerines

In [7]:
values = [255-20*n for n in range(len(classes))] 

In [8]:
pbar = ProgressBar()
for f in pbar(seg_coord):
    mask = np.zeros((2448,3264,3), dtype=np.uint8)
    for foods in seg_coord[f]:
        for fd in foods:
            
            w = foods[fd]['w']
            h = foods[fd]['h']
            r = np.array(w)
            c = np.array(h)
            rr, cc = polygon(r, c)
            
            mask[rr, cc] = values[classes.index(fd)]
            
    imageio.imwrite("C:/Users/Admin/Desktop/New folder (5)/datasets/food/test/"+f[:-4]+"AG"+".jpg",mask)


100% |########################################################################|


In [9]:
classes_en = ['bread','pasta','scallops','carrots','yogurt','pizza','cutlet','tangerines','potatoes/mashed potato','green beans', 'spinach','pudding']
#bread, pasta, carrots, pizza, tangerines

classes_wanted = ['bread', 'pasta', 'carrots', 'pizza', 'tangerines','potatoes/mashed potato']
#values_wanted = [values[classes_en.index(c)] for c in classes_wanted]
#values_unwanted = [v for v in values if v not in values_wanted]

values = [255, 235, 215, 195, 175, 155, 135, 115, 95, 75, 55, 35]
values_wanted = [255, 235, 195, 155, 115, 95]
values_unwanted = [215, 175, 135, 75, 55, 35]

In [ ]:
#remove unwanted segmentations
mp = "C:/Users/Admin/Desktop/New folder (5)/datasets/food/test_masks/"
masks = os.listdir(mp)

for m in masks:
    r = cv2.imread(mp+m)
    
    for vu in values_unwanted:
        r[(r>vu-20) & (r<vu+20)] = 0
    
    plt.figure(figsize=(5, 5))
    plt.axis('off')
    plt.imshow(r)
    
    #imageio.imwrite("C:/Users/Admin/Desktop/New folder (5)/datasets/food/test_masks_reduced/"+m,r)
    break